In [ ]:
#função para gerar valores aleatórios de -1 a 1 para usar no polinomio zernike
#Gera um único valor aleatório entre -1 e 1

function valorAleatorio = gerarAleatorioEntreAeB(a, b)
    % Esta função gera um valor aleatório entre a e b.

    % Verifica se a e b são inteiros ou decimais
    if ~isnumeric(a) || ~isnumeric(b)
        error('Os parâmetros devem ser valores numéricos.');
    end

    % Gera um valor aleatório entre a e b
    valorAleatorio = (b - a) * rand() + a;
end


In [ ]:
#função para gravar dados com um determinado caminho dado
##Certifique-se de substituir os caminhos 'C:\Caminho\Para\Salvar\Excel' e
##'C:\Caminho\Para\Salvar\PNG' pelos caminhos desejados onde você deseja salvar
##os arquivos Excel e PNG, respectivamente.

function salvarDados(valores, imageData2D, caminhoCSV, caminhoPNG)
    % Salva os valores em um arquivo CSV
    timestamp = datestr(now, 'yyyymmdd_HHMMSS');
    csvFilename = fullfile(caminhoCSV, ['saved_values_' timestamp '.csv']);
    writematrix(valores, csvFilename);
    disp(['Values saved to ' csvFilename '.']);

    % Salva a imagem imageData2D como um arquivo PNG
    pngFilename = fullfile(caminhoPNG, ['saved_image_' timestamp '.png']);
    imwrite(imageData2D, pngFilename);
    disp(['Image saved to ' pngFilename '.']);

    % Retorna uma mensagem indicando sucesso
    disp('Arquivos salvos com sucesso.');
end


In [ ]:
# insert before your code to establish connection to the camera

clear all
close all

NET.addAssembly([pwd, '\Thorlabs.TSI.TLCamera.dll']);
disp('Dot NET assembly loaded.');

tlCameraSDK = Thorlabs.TSI.TLCamera.TLCameraSDK.OpenTLCameraSDK;

serialNumbers = tlCameraSDK.DiscoverAvailableCameras;
disp([num2str(serialNumbers.Count), ' camera was discovered.']);

if (serialNumbers.Count > 0)

    disp('Opening the first camera')
    cam = tlCameraSDK.OpenCamera(serialNumbers.Item(0), false);

    gainRange = cam.GainRange;
    if (gainRange.Maximum > 0)
        cam.Gain = 0;
    end
end
cam.OperationMode = Thorlabs.TSI.TLCameraInterfaces.OperationMode.SoftwareTriggered;
cam.IssueSoftwareTrigger;
cam.ExposureTime_us = 300;  %define a large exposure to begin


# initizalizes the HoloSLM

cam.BitDepth
maxPixelValue = double(2^cam.BitDepth - 3);

H=1280; V=1024;
PixelSize = 12.5e-3;
x = PixelSize.*(-H/2:(H/2-1));
y = PixelSize.*(-V/2:(V/2-1));
[X,Y] = meshgrid(x,y);
[PHI,RHO] = cart2pol(X,Y);

Holo = zeros(V,H);
Holo_Blank = zeros(V,H);

# introducing zernike anomalies
counter = 1
while counter <= 1000

counter = counter + 1;

pist = 0;
x_tilt = 0;
y_tilt = 0;
h_comma = 0;
v_comma = 0;
spher = 0;

a = -1;
b = 1;

dfoc = gerarAleatorioEntreAeB(a, b);
v_astig = gerarAleatorioEntreAeB(a, b);
o_astig = gerarAleatorioEntreAeB(a, b);

HoloL = zernike(pist, x_tilt , y_tilt , dfoc, v_astig,  o_astig , h_comma ,v_comma , spher, H, V, RHO,PHI);

valores = [pist, x_tilt, y_tilt, dfoc, v_astig, o_astig, h_comma, v_comma, spher, H, V, RHO, PHI];

# Calibration
poly_file = fopen('polyfit_par_diode.txt','r');
formatSpec = '%f';
header = fgetl(poly_file);
polyvalues_cali = str2num(fgetl(poly_file));
fclose(poly_file);
GREYVAL = polyval(polyvalues_cali, mod(HoloL,2*pi));
GREYVAL = GREYVAL./255;
HoloSLM = [GREYVAL];
device_number = 2;
fullscreen(HoloSLM, device_number);

pause(1)
cam.MaximumNumberOfFramesToQueue = 1;
cam.FramesPerTrigger_zeroForUnlimited = 1;
cam.Arm;
cam.IssueSoftwareTrigger;

while (cam.NumberOfQueuedFrames == 0)
            pause(0.01);
        end
        imageFrame = cam.GetPendingFrameOrNull;
        if ~isempty(imageFrame)
            imageData = uint16(imageFrame.ImageData.ImageData_monoOrBGR);
            imageHeight = imageFrame.ImageData.Height_pixels;
            imageWidth = imageFrame.ImageData.Width_pixels;
            imageData2D = reshape(uint16(imageData), [imageWidth, imageHeight]);
            imageData2D(858:876, 398:429) = zeros(19,32);
            if 1
                fig3 = figure(3);
                fig3; imagesc(imageData2D'), colormap(gray), colorbar
            end
            % disp("ok")
        end

exposure = cam.ExposureTime_us;

caminhoCSV = '';
caminhoPNG = '';

salvarDados(valores, imageData2D, caminhoCSV, caminhoPNG);

disp('Releasing the camera');
cam.Dispose;
delete(cam);
delete(serialNumbers);
tlCameraSDK.Dispose;
delete(tlCameraSDK);

function  Holo_correc  = zernike( pist, x_tilt , y_tilt ,  dfoc ,ver_ast, ob_ast, h_coma , v_coma , spher, H,V, RHO,PHI)

    Holo_0 = (zeros(V,H)+1)*pist;
    Holo_1 = x_tilt * (RHO.*cos(mod(PHI,2*pi))) ;
    Holo_2 = y_tilt* (RHO.*sin(mod(PHI,2*pi)) );
    Holo_3 = dfoc * ((2* ( RHO.* RHO) -1)) ;
    Holo_4 = ver_ast * (RHO.* RHO.*cos(2 * mod(PHI,2*pi)));
    Holo_5 = ob_ast * (( RHO.* RHO).*sin(2 * mod(PHI,2*pi)));
    Holo_6 = h_coma * ((3*( RHO.* RHO)-2) .* RHO .* cos(mod(PHI,2*pi)));
    Holo_7 = v_coma * ((3*( RHO.* RHO)-2) .* RHO .* sin(mod(PHI,2*pi)));
    Holo_8 = spher * (6*(RHO.*RHO.*RHO.*RHO) - 6*(RHO.*RHO) + 1);


    Holo_correc =  Holo_0 + Holo_1 + Holo_2+ Holo_3 + Holo_4+ Holo_5+ Holo_6+ Holo_7+ Holo_8;
end

function salvarDados(valores, imageData2D, caminhoCSV, caminhoPNG)
    % Salva os valores em um arquivo CSV
    timestamp = datestr(now, 'yyyymmdd_HHMMSS');
    csvFilename = fullfile(caminhoCSV, ['saved_values_' timestamp '.csv']);
    writematrix(valores, csvFilename);
    disp(['Values saved to ' csvFilename '.']);

    % Salva a imagem imageData2D como um arquivo PNG
    pngFilename = fullfile(caminhoPNG, ['saved_image_' timestamp '.png']);
    imwrite(imageData2D, pngFilename);
    disp(['Image saved to ' pngFilename '.']);

    % Retorna uma mensagem indicando sucesso
    disp('Arquivos salvos com sucesso.');
end


function valorAleatorio = gerarAleatorioEntreAeB(a, b)
    % Esta função gera um valor aleatório entre a e b.

    % Verifica se a e b são inteiros ou decimais
    if ~isnumeric(a) || ~isnumeric(b)
        error('Os parâmetros devem ser valores numéricos.');
    end

    % Gera um valor aleatório entre a e b
    valorAleatorio = (b - a) * rand() + a;
end

